##CONDA

In [ ]:
# download the CONDA dataset
!mkdir datasets
!mkdir datasets/conda
!wget -P datasets/conda/ -c 'https://raw.githubusercontent.com/usydnlp/CONDA/main/data/CONDA_train.csv'
!wget -P datasets/conda/ -c 'https://raw.githubusercontent.com/usydnlp/CONDA/main/data/CONDA_valid.csv'
!wget -P datasets/conda/ -c 'https://raw.githubusercontent.com/usydnlp/CONDA/main/data/CONDA_test.csv'

--2021-08-24 22:12:37--  https://raw.githubusercontent.com/usydnlp/CONDA/main/data/CONDA_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2856782 (2.7M) [text/plain]
Saving to: ‘datasets/conda/CONDA_train.csv’

CONDA_train.csv     100%[===================>]   2.72M  --.-KB/s    in 0.02s   

2021-08-24 22:12:38 (145 MB/s) - ‘datasets/conda/CONDA_train.csv’ saved [2856782/2856782]

--2021-08-24 22:12:38--  https://raw.githubusercontent.com/usydnlp/CONDA/main/data/CONDA_valid.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Lengt

In [ ]:
import pandas as pd

# import the dataset into pandas dataframe
conda_train = pd.read_csv('datasets/conda/CONDA_train.csv')
conda_val = pd.read_csv('datasets/conda/CONDA_valid.csv')
conda_test = pd.read_csv('datasets/conda/CONDA_test.csv')
conda_train.head(6)

,Id,matchId,conversationId,utterance,chatTime,playerSlot,playerId,intentClass,slotClasses,slotTokens
0,11263,697,3193,wow!,76,0,ANTS IN MY EYES JOHNSON,O,O,"wow (O),"
1,13741,843,3809,WTF,1563,5,M.k,O,T,"WTF (T),"
2,22125,1412,6199,wpe wpe,2853,1,Acqua Ragia,O,O O,"wpe (O), wpe (O),"
3,6453,439,1875,hahaha,1038,0,juicebox,O,O,"hahaha (O),"
4,9644,601,2713,wtf,1661,5,KAIST.Shadows,O,T,"wtf (T),"
5,6208,433,1836,i cant [SEPA] play [SEPA] with 4 trash,1674,6,Vintage <3,E,P O SEPA O SEPA O O O,"i (P), cant (O), [SEPA] (SEPA), play (O), [SEP..."


In [ ]:
# check for any null items
print("train set:")
print(conda_train.isnull().sum())
print("\nval set:")
print(conda_val.isnull().sum())
print("\ntest set:")
print(conda_test.isnull().sum())

train set:
Id                  0
matchId             0
conversationId      0
utterance           7
chatTime            0
playerSlot          0
playerId            8
intentClass         0
slotClasses       834
slotTokens        834
dtype: int64

val set:
Id                  0
matchId             0
conversationId      0
utterance           1
chatTime            0
playerSlot          0
playerId            1
intentClass         0
slotClasses       268
slotTokens        268
dtype: int64

test set:
Id                0
matchId           0
conversationId    0
utterance         1
chatTime          0
playerSlot        0
playerId          3
dtype: int64


In [ ]:
# remove rows with nan values
conda_train.dropna(inplace=True)
conda_val.dropna(inplace=True)
conda_test.dropna(inplace=True)

In [ ]:
# create a function to convert the conda dataset into jointBERT input
def create_input_file(data, column, file_name):
    # create a list of the slot text and tokens
    texts = list(data[column])

    # create a list of slot labels
    slot_label = ['(C)','(D)','(O)','(P)','(S)','(T)','(SEPA)',
                  '(C),','(D),','(O),','(P),','(S),','(T),','(SEPA),']

    # extract the text from the slot text and tokens
    all_new_text = []
    for text in texts:
      try: split_text = text.split()
      except: print(text) 
      new_text = []
      for stext in split_text:
        if stext not in slot_label:
          new_text.append(stext)
      join_new_text = " ".join(new_text)
      all_new_text.append(join_new_text)

    # write the text into seq.in file for text input to jointBERT model
    with open(file_name, 'w') as f:
        for item in all_new_text:
            f.write("%s\n" % item)

In [ ]:
# create input files for the train set
!mkdir datasets/conda/train
create_input_file(conda_train, 'slotTokens', 'datasets/conda/train/seq.in')
create_input_file(conda_train, 'slotClasses', 'datasets/conda/train/seq.out')
create_input_file(conda_train, 'intentClass', 'datasets/conda/train/label')

In [ ]:
# create input files for the val set
!mkdir datasets/conda/val
create_input_file(conda_val, 'slotTokens', 'datasets/conda/val/seq.in')
create_input_file(conda_val, 'slotClasses', 'datasets/conda/val/seq.out')
create_input_file(conda_val, 'intentClass', 'datasets/conda/val/label')

In [ ]:
# create input files for the test set (there is no slot/intent labels for the test set)
!mkdir datasets/conda/test
create_input_file(conda_test, 'utterance', 'datasets/conda/test/sample_pred_in.txt')

In [ ]:
# create a function to convert the conda dataset into jointBERT input
def create_input_file2(data, column):
    # create a list of the slot text and tokens
    texts = list(data[column])

    # create a list of slot labels
    slot_label = ['(C)','(D)','(O)','(P)','(S)','(T)','(SEPA)',
                  '(C),','(D),','(O),','(P),','(S),','(T),','(SEPA),']

    # extract the text from the slot text and tokens
    all_new_text = []
    for text in texts:
      try: split_text = text.split()
      except: print(text) 
      new_text = []
      for stext in split_text:
        if stext not in slot_label:
          new_text.append(stext)
      join_new_text = " ".join(new_text)
      all_new_text.append(join_new_text)
    
    return all_new_text

In [ ]:
train_text = create_input_file2(conda_train, 'slotTokens')
train_text = ['BOS ' + text + ' EOS\t' for text in train_text]
train_text[:5]

['BOS wow EOS\t',
 'BOS WTF EOS\t',
 'BOS wpe wpe EOS\t',
 'BOS hahaha EOS\t',
 'BOS wtf EOS\t']

In [ ]:
train_slot = create_input_file2(conda_train, 'slotClasses')
train_slot[:5]

['O', 'T', 'O O', 'O', 'T']

In [ ]:
train_label = create_input_file2(conda_train, 'intentClass')
train_label[:5]

['O', 'O', 'O', 'O', 'O']

In [ ]:
result = list(zip(train_text, train_slot))
result = ["".join(text) for text in result]
result[:5]

['BOS wow EOS\tO',
 'BOS WTF EOS\tT',
 'BOS wpe wpe EOS\tO O',
 'BOS hahaha EOS\tO',
 'BOS wtf EOS\tT']

In [ ]:
result_w_intent = list(zip(result, train_label))
result_w_intent = [" ".join(text) for text in result_w_intent]
result_w_intent[:5]

['BOS wow EOS\tO O',
 'BOS WTF EOS\tT O',
 'BOS wpe wpe EOS\tO O O',
 'BOS hahaha EOS\tO O',
 'BOS wtf EOS\tT O']

In [ ]:
with open('datasets/conda/conda.train.w-intent.iob', 'w') as f:
  for item in result_w_intent:
    f.write("%s\n" % item)

with open('datasets/conda/conda.train.iob', 'w') as f:
  for item in result:
    f.write("%s\n" % item)

In [ ]:
val_text = create_input_file2(conda_val, 'slotTokens')
val_text = ['BOS ' + text + ' EOS\t' for text in val_text]
val_slot = create_input_file2(conda_val, 'slotClasses')
val_label = create_input_file2(conda_val, 'intentClass')

result = list(zip(val_text, val_slot))
result = ["".join(text) for text in result]
result_w_intent = list(zip(result, val_label))
result_w_intent = [" ".join(text) for text in result_w_intent]

with open('datasets/conda/conda.dev.w-intent.iob', 'w') as f:
  for item in result_w_intent:
    f.write("%s\n" % item)

with open('datasets/conda/conda.dev.iob', 'w') as f:
  for item in result:
    f.write("%s\n" % item)

##LOW

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# download the low dataset
!mkdir datasets/low

# train
id = '1-CnkX1f_OiAYq5LLc7W_Bxs2hO3aNBgy'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/low/train.csv')

# train label
id = '1-C8mhh9wbiHzAKodKKH3FKUOydqfAv9K'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/low/train_label.csv')

# val
id = '1-Cc6X5tMm6FpH1PsQWv_pZYqDbZ_GpCe'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/low/val.csv')

# val label
id = '1-1PuElSAl1dcJJxdB1cs2KvNf35NnaSc'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/low/val_label.csv')

# test
id = '1-Cl6P0nOJRErw0BmzQ_OLWPTcbCczFMo'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/low/test.csv')

# test label
id = '1-40KJhMjrv7Yxkk-A6NYEMkbPl9sjv_4'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/low/test_label.csv')

In [ ]:
import pandas as pd

# import the dataset into pandas dataframe
train_text = pd.read_csv('datasets/low/train.csv', index_col=0)
train_label = pd.read_csv('datasets/low/train_label.csv', index_col=0)
val_text = pd.read_csv('datasets/low/val.csv', index_col=0)
val_label = pd.read_csv('datasets/low/val_label.csv', index_col=0)
test_text = pd.read_csv('datasets/low/test.csv', index_col=0)
test_label = pd.read_csv('datasets/low/test_label.csv', index_col=0)

In [ ]:
train_text.head()

,0,1,2,3,4,5,6,7,8,9,10,Unnamed: 12
13430,89498,nothinh come :D,enemy,00:07:29,8,2.0,0.0,Negative Attitude,1423.0,Nasus,O O O,NaN
28658,191675,"malph, would you gank bot?",enemy,00:08:58,4,1.0,0.0,Offensive Language,1973.0,Caitlyn,O O P S O,NaN
20418,135440,no flame just rage,offender,00:07:55,7,2.0,1.0,Verbal Abuse,1666.0,Gragas,O O O O,NaN
19964,133016,i do,enemy,00:35:16,7,2.0,0.0,Negative Attitude,1654.0,Warwick,P O,NaN
4446,22364,no mana :/,enemy,00:07:08,6,0.0,2.0,Negative Attitude,107.0,Ezreal,O S O,NaN


In [ ]:
train_label.head()

,Label
13430,O
28658,O
20418,A
19964,O
4446,O


In [ ]:
print(train_text.shape)
print(train_label.shape)

(29399, 12)
(29399, 1)


In [ ]:
# concat the text and the label files
train_data = pd.concat([train_text,train_label],axis=1,join='inner')
val_data = pd.concat([val_text,val_label],axis=1,join='inner')
test_data = pd.concat([test_text,test_label],axis=1,join='inner')

# drop the 'Unnamed: 12' column
train_data.drop(['Unnamed: 12'], axis=1, inplace=True)
val_data.drop(['Unnamed: 12'], axis=1, inplace=True)
test_data.drop(['Unnamed: 12'], axis=1, inplace=True)
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

(29399, 12)
(3267, 12)
(3630, 12)


In [ ]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,10,Label
13430,89498,nothinh come :D,enemy,00:07:29,8,2.0,0.0,Negative Attitude,1423.0,Nasus,O O O,O
28658,191675,"malph, would you gank bot?",enemy,00:08:58,4,1.0,0.0,Offensive Language,1973.0,Caitlyn,O O P S O,O
20418,135440,no flame just rage,offender,00:07:55,7,2.0,1.0,Verbal Abuse,1666.0,Gragas,O O O O,A
19964,133016,i do,enemy,00:35:16,7,2.0,0.0,Negative Attitude,1654.0,Warwick,P O,O
4446,22364,no mana :/,enemy,00:07:08,6,0.0,2.0,Negative Attitude,107.0,Ezreal,O S O,O


In [ ]:
# check for any null items
print("train set:")
print(train_data.isnull().sum())
print("\nval set:")
print(val_data.isnull().sum())
print("\ntest set:")
print(test_data.isnull().sum())

train set:
0         0
1        14
2         0
3         0
4         0
5         1
6         1
7         1
8         1
9         1
10        1
Label     1
dtype: int64

val set:
0        0
1        5
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
Label    0
dtype: int64

test set:
0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
Label    0
dtype: int64


In [ ]:
# remove rows with nan values
train_data.dropna(inplace=True)
val_data.dropna(inplace=True)
print("train set:")
print(train_data.isnull().sum())
print("\nval set:")
print(val_data.isnull().sum())

train set:
0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
Label    0
dtype: int64

val set:
0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
Label    0
dtype: int64


In [ ]:
# delete texts that contain '#ERROR!'
train_data.drop(train_data['1'][train_data['1']=='#ERROR!'].index, inplace=True)
val_data.drop(val_data['1'][val_data['1']=='#ERROR!'].index, inplace=True)
test_data.drop(test_data['1'][test_data['1']=='#ERROR!'].index, inplace=True)

In [ ]:
# check the intent labels
print(set(train_data['Label']))
print(set(val_data['Label']))
print(set(test_data['Label']))

{'I ', 'E', 'I', 'O', 'A'}
{'T', 'E', 'I', 'O', 'A'}
{'A', 'E', 'O', 'I'}


In [ ]:
# remove the val data with 'T' intent label
val_data.drop(val_data['Label'][val_data['Label']=='T'].index, inplace=True)

In [ ]:
# 'I' and 'I ' intent labels in train set should be the same as 'I'
labels = [label.strip().upper() for label in list(train_data['Label'])]
train_data['Label']=labels
print(set(train_data['Label']))
print(set(val_data['Label']))
print(set(test_data['Label']))

{'A', 'E', 'O', 'I'}
{'A', 'E', 'O', 'I'}
{'A', 'E', 'O', 'I'}


In [ ]:
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

(29370, 12)
(3259, 12)
(3630, 12)


In [ ]:
# create a function to remove rows in which the length of the text != the length of the slot tokens
def remove_error(data):
    text_slot = list(zip(list(data['1']),list(data['10'])))

    to_remove = []
    for i, text in enumerate(text_slot):
      if len(str(text[0]).split())!=len(text[1].split()):
        to_remove.append(i)
        print("index:",i)
        print("text:",text[0])
        print("slot label:",text[1])

    to_remove_index = [data.iloc[[n]].index[0] for n in to_remove]
    data.drop(to_remove_index, inplace=True)
    print(data.shape)

In [ ]:
# remove rows in train set 
remove_error(train_data)

index: 3301
text:  ''''''''''''' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'p dfd
slot label: O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O
index: 11968
text:  ' '
slot label: O O O
index: 12918
text:  '
slot label: O O
index: 13366
text:  '
slot label: O O
index: 15245
text:  what¿?
slot label: O O
index: 16196
text: I
slot label: S O
index: 18916
text: I
slot label: S L O O
index: 20506
text: O
slot label: S S
index: 20547
text:  ' ' ' ' ' ' '
slot label: O O O O O O O O
index: 26000
text: I
slot label: O P O L
index: 27479
text: E
slot label: O S L S
index: 27966
text: I
slot label: O O O O P S O L S S O
(29358, 12)


In [ ]:
# remove rows in val set 
remove_error(val_data)

index: 1458
text: I
slot label: O O
(3258, 12)


In [ ]:
# remove rows in test set 
remove_error(test_data)

index: 1815
text:  ' '
slot label: O O O
index: 2164
text:  ' '
slot label: O O O
(3628, 12)


In [ ]:
# check the proportion of intent labels in the dataset
print("train E:",train_data[train_data['Label']=='E']['Label'].count()/train_data.shape[0])
print("train I:",train_data[train_data['Label']=='I']['Label'].count()/train_data.shape[0])
print("train A:",train_data[train_data['Label']=='A']['Label'].count()/train_data.shape[0])
print("train O:",train_data[train_data['Label']=='O']['Label'].count()/train_data.shape[0])
print("train_data length:",train_data.shape[0])
print()
print("val E:",val_data[val_data['Label']=='E']['Label'].count()/val_data.shape[0])
print("val I:",val_data[val_data['Label']=='I']['Label'].count()/val_data.shape[0])
print("val A:",val_data[val_data['Label']=='A']['Label'].count()/val_data.shape[0])
print("val O:",val_data[val_data['Label']=='O']['Label'].count()/val_data.shape[0])
print("val_data length:",val_data.shape[0])
print()
print("test E:",test_data[test_data['Label']=='E']['Label'].count()/test_data.shape[0])
print("test I:",test_data[test_data['Label']=='I']['Label'].count()/test_data.shape[0])
print("test A:",test_data[test_data['Label']=='A']['Label'].count()/test_data.shape[0])
print("test O:",test_data[test_data['Label']=='O']['Label'].count()/test_data.shape[0])
print("test_data length:",test_data.shape[0])

train E: 0.06727297499829689
train I: 0.00991211935417944
train A: 0.02503576537911302
train O: 0.8977791402684107
train_data length: 29358

val E: 0.08011049723756906
val I: 0.008287292817679558
val A: 0.027317372621240024
val O: 0.8842848373235114
val_data length: 3258

test E: 0.06560088202866593
test I: 0.009095920617420067
test A: 0.027839029768467475
test O: 0.8974641675854466
test_data length: 3628


In [ ]:
# create a function to convert the low dataset into jointBERT input
def create_input(data,set):
    seq_in = list(data['1'])
    with open('datasets/low/'+set+'/seq.in', 'w') as f:
        for item in seq_in:
            f.write("%s\n" % item)

    seq_out = list(data['10'])
    with open('datasets/low/'+set+'/seq.out', 'w') as f:
        for item in seq_out:
            f.write("%s\n" % item)

    label = list(data['Label'])
    with open('datasets/low/'+set+'/label', 'w') as f:
        for item in label:
            f.write("%s\n" % item)

In [ ]:
# create the directory
!mkdir datasets/low/train
!mkdir datasets/low/val
!mkdir datasets/low/test

# create the input files for jointBERT
create_input(train_data,'train')
create_input(val_data,'val')
create_input(test_data,'test')

In [ ]:
data = train_data
seq_in = list(data['1'])
seq_out = list(data['10'])
label = list(data['Label'])
seq_in = ['BOS ' + text + ' EOS\t' for text in seq_in]
result = list(zip(seq_in, seq_out))
result = ["".join(text) for text in result]
result_w_intent = list(zip(result, label))
result_w_intent = [" ".join(text) for text in result_w_intent]

with open('datasets/low/low.train.w-intent.iob', 'w') as f:
  for item in result_w_intent:
    f.write("%s\n" % item)

with open('datasets/low/low.train.iob', 'w') as f:
  for item in result:
    f.write("%s\n" % item)

In [ ]:
data = val_data
seq_in = list(data['1'])
seq_out = list(data['10'])
label = list(data['Label'])
seq_in = ['BOS ' + text + ' EOS\t' for text in seq_in]
result = list(zip(seq_in, seq_out))
result = ["".join(text) for text in result]
result_w_intent = list(zip(result, label))
result_w_intent = [" ".join(text) for text in result_w_intent]

with open('datasets/low/low.dev.w-intent.iob', 'w') as f:
  for item in result_w_intent:
    f.write("%s\n" % item)

with open('datasets/low/low.dev.iob', 'w') as f:
  for item in result:
    f.write("%s\n" % item)

In [ ]:
data = test_data
seq_in = list(data['1'])
seq_out = list(data['10'])
label = list(data['Label'])
seq_in = ['BOS ' + text + ' EOS\t' for text in seq_in]
result = list(zip(seq_in, seq_out))
result = ["".join(text) for text in result]
result_w_intent = list(zip(result, label))
result_w_intent = [" ".join(text) for text in result_w_intent]

with open('datasets/low/low.test.w-intent.iob', 'w') as f:
  for item in result_w_intent:
    f.write("%s\n" % item)

with open('datasets/low/low.test.iob', 'w') as f:
  for item in result:
    f.write("%s\n" % item)

##Additional embeddings

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate
drive = None
def authenticate():
  global drive
  
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

#Download files
def downloadFiles(fileIds):
  authenticate()
  
  for fileId in fileIds:    
    
    downloaded = drive.CreateFile({"id": fileId[1]})
    downloaded.GetContentFile(fileId[0])

#Download file if not existing
try:
  _ = open("RNN-for-Joint-NLU-master/README.md", "r")
except:
  downloadFiles([["RNN-for-Joint-NLU-master.zip", "1S3Ojq4WOa3kDTgZaSWi-ASymYaqq-spU"]])
  !unzip "RNN-for-Joint-NLU-master.zip" 

# Download file if not existing
try:
  _ = open("RNN-for-Joint-NLU-master/dataset/conda/conda.dev.iob", "r")
except:
  downloadFiles([["conda_low_dataset2.zip", "1iUNzJBB0MF6eN0ySGQCJNHnk7kb_uiqk"]])
  !unzip "conda_low_dataset2.zip" -d "RNN-for-Joint-NLU-master/dataset/"

In [2]:
def data_pipeline(data):
    data = [t[:-1] for t in data] 
    data = [[t.split("\t")[0].split(" "), t.split("\t")[1].split(" ")[:-1], t.split("\t")[1].split(" ")[-1]] for t in
            data]
    data = [[t[0][1:-1], t[1][:], t[2]] for t in data] 
    seq_in, seq_out, intent = list(zip(*data))

    return seq_in, seq_out, intent

CONDA only

In [3]:
train_data =  open("./RNN-for-Joint-NLU-master/dataset/conda/conda.train.w-intent.iob", "r").readlines()
dev_data = open("./RNN-for-Joint-NLU-master/dataset/conda/conda.dev.w-intent.iob", "r").readlines()
test_data = dev_data
print("#"*100)
print("Training data: "+str(len(train_data)))
print("Development data: "+str(len(dev_data)))
print("Testing data: "+str(len(test_data)))
print("#"*100)

train_seq, train_slot, train_intent = data_pipeline(train_data)
dev_seq, dev_slot, dev_intent = data_pipeline(dev_data)
test_seq, test_slot, test_intent = data_pipeline(test_data)

print("#"*100)
print("Data example:")
print("Sequence: ")
print(train_seq[0])
print()
print("Intent: ")
print(train_intent[0])
print()
print("Slots: ")
print(train_slot[0])
print("#"*100)

####################################################################################################
Training data: 26078
Development data: 8705
Testing data: 8705
####################################################################################################
####################################################################################################
Data example:
Sequence: 
['wow']

Intent: 
O

Slots: 
['O']
####################################################################################################


LOL only

In [3]:
train_data =  open("./RNN-for-Joint-NLU-master/dataset/low/low.train.w-intent.iob", "r").readlines()
dev_data=open("./RNN-for-Joint-NLU-master/dataset/low/low.dev.w-intent.iob", "r").readlines()
test_data = open("./RNN-for-Joint-NLU-master/dataset/low/low.test.w-intent.iob", "r").readlines()
print("#"*100)
print("Training data: "+str(len(train_data)))
print("Development data: "+str(len(dev_data)))
print("Testing data: "+str(len(test_data)))
print("#"*100)

train_seq, train_slot, train_intent = data_pipeline(train_data)
dev_seq, dev_slot, dev_intent = data_pipeline(dev_data)
test_seq, test_slot, test_intent = data_pipeline(test_data)

print("#"*100)
print("Data example:")
print("Sequence: ")
print(train_seq[0])
print()
print("Intent: ")
print(train_intent[0])
print()
print("Slots: ")
print(train_slot[0])
print("#"*100)

####################################################################################################
Training data: 29358
Development data: 3258
Testing data: 3628
####################################################################################################
####################################################################################################
Data example:
Sequence: 
['nothinh', 'come', ':D']

Intent: 
O

Slots: 
['O', 'O', 'O']
####################################################################################################


In [4]:
# remove ''
for seq in train_seq:
  while '' in seq:
    seq.remove('')

for seq in dev_seq:
  while '' in seq:
    seq.remove('')

for seq in test_seq:
  while '' in seq:
    seq.remove('')
train_seq[:2]

(['nothinh', 'come', ':D'], ['malph,', 'would', 'you', 'gank', 'bot?'])

Continue for both

In [5]:
vocab=set()
max_len=0

for seq_train in train_seq:

  if (len(seq_train)>max_len):
    max_len=len(seq_train)
  for word in seq_train:
    vocab.add(word)
    
for seq_dev in dev_seq:
  if (len(seq_dev)>max_len):  
    max_len=len(seq_dev)
  for word in seq_dev:
     vocab.add(word)  
    
for seq_test in test_seq:
  if (len(seq_test)>max_len):
     max_len=len(seq_test)
  for word in seq_test:
     vocab.add(word)
  
temp=['<PAD>']
temp.extend(list(vocab))
vocab=temp
VOCAB_SIZE=len(vocab)    
MAX_SEQ_LENGTH=max_len  

slot_all=set()
intent_all=set()

for train_sequ, train_int in zip(train_slot, train_intent):
  intent_all.add(train_int)
  for i in train_sequ:
    slot_all.add(i)
for dev_sequ, dev_int in zip(dev_slot, dev_intent):
  intent_all.add(dev_int)
  for i in dev_sequ:
    slot_all.add(i)
for test_sequ, test_int in zip(test_slot, test_intent):
  intent_all.add(test_int)
  for i in test_sequ:
    slot_all.add(i)  
temp=['<PAD>']
temp.extend(list(slot_all))
#slot_all.add('<PAD>')
#slot_all.add('[SEP]')
slot_all=temp
intent_all=list(intent_all)

slot_num=len(slot_all)
intent_num=len(intent_all)
print("#"*85)
print("Vocab size: " + str(VOCAB_SIZE))
print("Max sentence length: "+ str(MAX_SEQ_LENGTH))
print("Total slots: " + str(slot_num))
print("Total intents: " + str(intent_num))
print("#"*85)
print(list(vocab)[-5:])
print(list(slot_all)[-5:])
print(list(intent_all)[-5:])
print("#"*85)

#####################################################################################
Vocab size: 17784
Max sentence length: 124
Total slots: 7
Total intents: 4
#####################################################################################
['offensiv', 'nice...', 'fauk', 'chouf', 'msot']
['P', 'C', 'L', 'S', 'T']
['I', 'A', 'E', 'O']
#####################################################################################


In [6]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np

import re

tokenized_words = train_seq + dev_seq
label_list = train_intent + dev_intent

length_list = [len(words) for words in tokenized_words]	
print(max(length_list))
seq_length = 18

# word index encode
from keras.preprocessing.sequence import pad_sequences

tokenPAD = "<pad>"
tokenUNK = "<unknown>"

unique_words_temp = set()
for sentence in tokenized_words:
    for word in sentence:
        unique_words_temp.add(word.lower())
unique_words = [tokenPAD, tokenUNK]
unique_words.extend(unique_words_temp)

unique_words_dict = {word: i for i, word in enumerate(unique_words)}

word_index_encode = []
for sentence in tokenized_words:
    word_index_encode_temp = []    
    for word in sentence:
        try:
            word_index_encode_temp.append(unique_words_dict[word])
        except KeyError:
            word_index_encode_temp.append(unique_words_dict[tokenUNK])
    word_index_encode.append(word_index_encode_temp)

#Pad sequences
word_index_encode = pad_sequences(word_index_encode, maxlen = seq_length, value = unique_words_dict[tokenPAD], padding = "post")

#label encoding
from sklearn.preprocessing import LabelEncoder

labels = np.unique(label_list)

num_class = len(labels)
lEnc = LabelEncoder()
lEnc.fit(labels)

encoded_labels = lEnc.transform(label_list)
print(labels)
print(lEnc.transform(labels))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
124
['A' 'E' 'I' 'O']
[0 1 2 3]


###CONDA and LOL additional embeddings

In [7]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# create directory
!mkdir datasets

# download and unzip files
id = '1u5H0LCc4WLgXdj-JVGbJMyZ3M4w6PkaD'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('proper_jbert_embeddings.zip')
!unzip proper_jbert_embeddings.zip

# download and unzip files
id = '1zhosfLkz3QuzSYYdTxQfV9FQpt-yK-QK'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/conda_lol_dg_embeddings.zip')
!unzip datasets/conda_lol_dg_embeddings.zip -d datasets/

Archive:  proper_jbert_embeddings.zip
  inflating: proper_jbert_ner_embeddings/CONDA_dev_NER_embedding.npy  
  inflating: proper_jbert_ner_embeddings/CONDA_train_NER_embedding.npy  
  inflating: proper_jbert_ner_embeddings/LOL_dev_NER_embedding.npy  
  inflating: proper_jbert_ner_embeddings/LOL_test_NER_embedding.npy  
  inflating: proper_jbert_ner_embeddings/LOL_train_NER_embedding(1).npy  
Archive:  datasets/conda_lol_dg_embeddings.zip
  inflating: datasets/conda_glove_embed.pkl  
   creating: datasets/jbert/conda/
  inflating: datasets/jbert/conda/CONDA_dev_NER_embedding.npy  
  inflating: datasets/jbert/conda/CONDA_train_NER_embedding.npy  
   creating: datasets/jbert/lol/
  inflating: datasets/jbert/lol/LOL_dev_NER_embedding.npy  
  inflating: datasets/jbert/lol/LOL_test_NER_embedding.npy  
  inflating: datasets/jbert/lol/LOL_train_NER_embedding.npy  
   creating: datasets/transformers/conda/
  inflating: datasets/transformers/conda/CONDA_dev_NER_embedding.npy  
  inflating: datas

In [9]:
import numpy as np
#nerembed_train = np.load('proper_jbert_ner_embeddings/CONDA_train_NER_embedding.npy')
#nerembed_test = np.load('proper_jbert_ner_embeddings/CONDA_dev_NER_embedding.npy')
nerembed_train = np.load('proper_jbert_ner_embeddings/LOL_train_NER_embedding(1).npy')
nerembed_dev = np.load('proper_jbert_ner_embeddings/LOL_dev_NER_embedding.npy')
nerembed_test = np.load('proper_jbert_ner_embeddings/LOL_test_NER_embedding.npy')
print(nerembed_train.shape)
print(nerembed_dev.shape)
print(nerembed_test.shape)

(29358, 50, 9)
(3258, 50, 9)
(3628, 50, 9)


In [10]:
#nerembed_all = np.concatenate((nerembed_train, nerembed_test), axis=0)
nerembed_all = np.concatenate((nerembed_train, nerembed_dev, nerembed_test), axis=0)
nerembed_all.shape

(36244, 50, 9)

In [11]:
nerembed_nozero = np.array([np.array([item for item in sent if (item!=0).all()]) for sent in nerembed_all])
nerembed_nozero.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


(36244,)

In [12]:
emb_len = [len(item) for item in nerembed_nozero]
emb_len[:5]

[7, 11, 6, 4, 6]

In [13]:
# install
!pip install pytorch-pretrained-bert pytorch-nlp

# BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertModel
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [14]:
print(train_seq[:10])
print(dev_seq[:10])
print(test_seq[:10])

(['nothinh', 'come', ':D'], ['malph,', 'would', 'you', 'gank', 'bot?'], ['no', 'flame', 'just', 'rage'], ['i', 'do'], ['no', 'mana', ':/'], ['like', 'soraka'], ['np'], ['Its', 'my', 'fault', 'really...'], ['coming'], ['ok'])
(['Y', 'U', 'DO', 'DIS'], ['omg', 'i', 'think', 'you', 'main', 'supp', 'an', 'dadc'], ["you'll", 'get', 'gold'], ['you', 'dont', 'deserve', 'it'], ['i', 'dont', 'care', 'stop', 'ping', 'me'], ['boar', 'that', 'connection', 'problems', '-.-'], ['WORST', 'jUNGLER', 'EVER'], ['if', 'i', 'get', 'more', 'kills', 'i', 'carry', 'this', 'niggaa'], [';D'], ['i', 'cry'])
(['guys', 'together'], ['how', 'dare', 'we', 'go', '0-2', 'feeders', 'bronze', 'kids'], ['gj', 'retard'], ['yup', 'ignite', 'to', 'ks', 'noob', 'act'], ['NO', 'FLASH', 'THRESH'], ['puss', 'y', 'ez'], ['wp', 'reprot', 'shaco'], ['4', 'bot', "can't", 'do', 'sht'], ['where', 'is', 'BULWARK?'], ['they', 'get', '1', 'kill'])


In [15]:
# Tokenize with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

sentences = []
#for query in train_seq + dev_seq:
for query in train_seq + dev_seq + test_seq:
  tokens = []
  tokens.append('[CLS]')
  for word in query:
      word_tokens = tokenizer.tokenize(word)
      if not word_tokens:
          word_tokens = '[UNK]'  # For handling the bad-encoded word
      tokens.extend(word_tokens)
  tokens.append('[SEP]')
  sentences.append(tokens)
print(sentences)

[['[CLS]', 'nothin', '##h', 'come', ':', 'd', '[SEP]'], ['[CLS]', 'mal', '##ph', ',', 'would', 'you', 'gan', '##k', 'bot', '?', '[SEP]'], ['[CLS]', 'no', 'flame', 'just', 'rage', '[SEP]'], ['[CLS]', 'i', 'do', '[SEP]'], ['[CLS]', 'no', 'mana', ':', '/', '[SEP]'], ['[CLS]', 'like', 'so', '##rak', '##a', '[SEP]'], ['[CLS]', 'np', '[SEP]'], ['[CLS]', 'its', 'my', 'fault', 'really', '.', '.', '.', '[SEP]'], ['[CLS]', 'coming', '[SEP]'], ['[CLS]', 'ok', '[SEP]'], ['[CLS]', 'we', 'me', '##ed', 'a', 'mage', '[SEP]'], ['[CLS]', 'vi', 'mid', 'vi', '[SEP]'], ['[CLS]', 'yeah', '[SEP]'], ['[CLS]', 'run', '##ned', '?', 'right', '?', 'i', 'had', '-', '-', '-', 'lp', 'you', 'fool', '.', '[SEP]'], ['[CLS]', 'om', '##w', '[SEP]'], ['[CLS]', 'hard', 'to', 'lane', 'vs', 'tee', '##mo', '[SEP]'], ['[CLS]', 'same', 'so', 'many', 'trolls', 'in', 'this', 'game', '[SEP]'], ['[CLS]', 'and', 'that', 'is', 'what', 'happens', 'when', 'you', 'get', 'the', 'phone', '[SEP]'], ['[CLS]', 'feed', '##ar', '[SEP]'], ['[CL

In [17]:
print(len(sentences))
print(len(train_seq))
print(len(dev_seq))
print(len(test_seq))

36244
29358
3258
3628


In [18]:
max_seq_len = 50 # 18 for conda and 28 for lol tranformer models
vocab_ = set()
all_vocab = {}
for n,sent in enumerate(sentences): # for jbert
#for n,sent in enumerate(tokenized_words): # for transformer models
  for m,word in enumerate(sent[:max_seq_len]):
    if word not in vocab_:
      all_vocab[word] = nerembed_nozero[n][m]
      vocab_.add(word)
len(all_vocab)

5931

In [19]:
all_vocab['[CLS]']

array([ 0.14663459, -0.05262458, -0.00995038, -0.09029851,  0.12263373,
       -0.14626154,  0.02043098, -0.1690678 ,  0.03294891])

In [20]:
import pickle

# function to save the features
def save_features(feature,file_name):
    with open(file_name, 'wb') as fp:
        pickle.dump(feature, fp)

#save_features(all_vocab,'conda_ner_embed.pkl')
save_features(all_vocab,'lol_ner_embed.pkl')

In [ ]:
import pickle

gpgn_glove_dict = pickle.load( open("datasets/gpgn_glove_dict.pkl", "rb" ) )
glove_embedding_matrix = np.zeros((len(unique_words), 300))
for i, word in enumerate(unique_words):
    try:
        if word != tokenPAD: #set pad weights to 0
            glove_embedding_matrix[i] = gpgn_glove_dict[word]
    except KeyError:
        pass
        
glove_embedding_matrix.shape

(14669, 300)

In [ ]:
glove_embed = {word: glove_embedding_matrix[i] for i, word in enumerate(unique_words)}
print(len(glove_embed))

14669


In [ ]:
list(glove_embed.items())[:10]

[('<pad>',
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [ ]:
#save_features(glove_embed,'conda_glove_embed.pkl')
save_features(glove_embed,'lol_glove_embed.pkl')

###Waseem

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# download the low dataset
!mkdir datasets
!mkdir datasets/waseem

# dataset
id = '1gNfBqguzBu8cHKMPc8C44GbvD443dNC5'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/waseem/waseem.csv')

# direct NER
try:
  id = '1EbyrtoRp5Z2_sxcok_jbbFm1D8SWRXLK'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/waseem/NER_embedding.npy')
except:
  print("direct NER not working")

# Generalize: GP GN GLOVE
try:
  id = '15z7eDo1hPg_NCTR5HzoQZSRvFbQUkpDZ'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/waseem/gpgn_embedding.npy')
except:
  print("Generalize: GP GN GLOVE not working")

# Explicit: dict2vec
try: 
  id = '1UOAXT8Sl-NhvWdVCxd64gz9q6x37Wo6n'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/waseem/dict2vec_embedding.npy')
except:
  print("Explicit: dict2vec not working")

# Implicit: Sarcasm
try:
  id = '1-A2OU0VraTwJksmD0-TJWq5Fkihq-nTp'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/waseem/sarcasm_embedding.txt')
except:
  print("Implicit: Sarcasm not working")

direct NER not working
Generalize: GP GN GLOVE not working
Explicit: dict2vec not working


###Davidson

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# download the low dataset
!mkdir datasets
!mkdir datasets/davidson

# dataset
id = '1U6mBF-3tLYHUeHITWAu1pWpP3pHuVn9r'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/davidson/davidson.csv')

# direct NER
try:
  id = '1TEkLEP7ffQkVAU2C257sjMgacIAQdwdo'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/davidson/NER_embedding.npy')
except:
  print("direct NER not working")

# Generalize: GP GN GLOVE
try:
  id = '1wONGsnJbgkVzpBUkwLAli6P0LXFPUHYB'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/davidson/gpgn_embedding.npy')
except:
  print("Generalize: GP GN GLOVE not working")

# Explicit: dict2vec
try: 
  id = '1pz_cZQ7yIuTvhavkEnUaMEPmIurTBmD1'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/davidson/dict2vec_embedding.npy')
except:
  print("Explicit: dict2vec not working")

# Implicit: Sarcasm
try:
  id = '1uHPZSVdPslQOfHIJ6NlvIRK5bgXXeGKq'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/davidson/sarcasm_embedding.txt')
except:
  print("Implicit: Sarcasm not working")

mkdir: cannot create directory ‘datasets’: File exists
direct NER not working
Generalize: GP GN GLOVE not working
Explicit: dict2vec not working


###Fox

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# download the low dataset
!mkdir datasets
!mkdir datasets/fox

# dataset
id = '1fKnmVJYBVkLKSr6YDXwUz5tZAHDc7YvO'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/fox/fox.csv')

# direct NER
try:
  id = '1v8HLEbs2cZzjoDCxZ1wim6SLg714Ostw'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/fox/NER_embedding.npy')
except:
  print("direct NER not working")

# Generalize: GP GN GLOVE
try:
  id = '1WZPQkC7YnwySMbbyGb0IfqSUFg5NEbI2'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/fox/gpgn_embedding.npy')
except:
  print("Generalize: GP GN GLOVE not working")

# Explicit: dict2vec
try: 
  id = '1x0Q2r7s6BaXV2JGReNGnut5-ACiHISN1'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/fox/dict2vec_embedding.npy')
except:
  print("Explicit: dict2vec not working")

# Implicit: Sarcasm
try:
  id = '1ckvxrsTcyR6isINWhEDapBBYxJs4K03e'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/fox/sarcasm_embedding.txt')
except:
  print("Implicit: Sarcasm not working")

mkdir: cannot create directory ‘datasets’: File exists


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/__init__.py", line 44, in autodetect
    from . import file_cache
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    "file_cach

In [ ]:
import pandas as pd
df = pd.read_csv('datasets/fox/fox.csv')
df.head()

,Unnamed: 0,title,text,label,succ,meta,user,mentions,prev
0,0,German lawmakers approve 'no means no' rape la...,Merkel would never say NO,hateful,NaN,German lawmakers passed a bill Thursday that w...,barryswallows,NaN,NaN
1,1,German lawmakers approve 'no means no' rape la...,"Expect more and more women to be asking .. ""wh...",hateful,NaN,German lawmakers passed a bill Thursday that w...,PostApocalypticHero,NaN,NaN
2,2,German lawmakers approve 'no means no' rape la...,Groping people in public wasn't already illega...,non-hateful,NaN,German lawmakers passed a bill Thursday that w...,californiamojo,NaN,NaN
3,3,German lawmakers approve 'no means no' rape la...,"Merkel, possible the only person in charge who...",hateful,NaN,German lawmakers passed a bill Thursday that w...,MikeSte,NaN,NaN
4,4,German lawmakers approve 'no means no' rape la...,"They know very well, no means NO! They need to...",hateful,NaN,German lawmakers passed a bill Thursday that w...,scientist,NaN,NaN


In [ ]:
df.shape

(1526, 9)

In [ ]:
df['text'][0]

'Merkel would never say NO'

In [ ]:
df['title'][0]

"German lawmakers approve 'no means no' rape law after Cologne assaults"

In [ ]:
len(df['meta'][0])

178

In [ ]:
import numpy as np
fox_nerembed = np.load('datasets/fox/NER_embedding.npy')
print(fox_nerembed.shape)

(1526, 431, 9)


In [ ]:
fox_nerembed[0]

array([[ -4.02332592,  -1.39931381,   0.33014968, ...,  -3.41104174,
         -3.92649555,  -4.33968449],
       [ -6.91270208,  -5.5939126 ,  -4.13517618, ...,  -8.49687004,
         -9.45039177,  -7.81197166],
       [ -7.72759438,  -7.16251087,  -2.19979548, ...,  -6.90222549,
         -7.85770655, -10.30075932],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [ ]:
fox_unbiglvembed = np.load('datasets/fox/gpgn_embedding.npy')
print(fox_unbiglvembed.shape)

(1526, 431, 300)


In [ ]:
fox_unbiglvembed[:,:18,:]

array([[[-0.10422102,  0.06783178, -0.11520538, ...,  0.04039161,
         -0.0854458 , -0.24887985],
        [-0.07318079,  0.10413598,  0.21059963, ..., -0.14471748,
         -0.3566629 ,  0.14755233],
        [ 0.03479033,  0.0177833 ,  0.32607347, ..., -0.33580911,
         -0.31246632, -0.09985618],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[-0.30182382,  0.33181989, -0.10680452, ..., -0.16449349,
         -0.31168154, -0.00599835],
        [-0.03334049,  0.2958872 , -0.03110489, ...,  0.10472272,
         -0.18761432,  0.21740013],
        [ 0.12876447, -0.26626369,  0.05919686, ..., -0.04277867,
          0.01490089, -0.10107212],
        ...,
        [-0.0667833 ,  0.16519065,  0.51832473, ..., -

In [ ]:
fox_dict2vecembed = np.load('datasets/fox/dict2vec_embedding.npy')
print(fox_dict2vecembed.shape)

(1526, 431, 300)


In [ ]:
sarcasm_embedding_list = []

with open("datasets/fox/sarcasm_embedding.txt","r") as f:
  raw_string = f.readline()
  raw_string = raw_string[1:-1]
  
  raw_lists = raw_string.split('[')
  for l in raw_lists[1:]:
    l = l[:-3]
    float_list = l.split(",")
    float_list = [float(f) for f in float_list]
    sarcasm_embedding_list.append(float_list)
sarcasm_embedding=np.array(sarcasm_embedding_list)   
del sarcasm_embedding_list 
print(sarcasm_embedding.shape)

(1526, 2)


In [ ]:
sarcasm_embedding[0]

array([ 2.640789, -3.651072])

###OffensEval

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# download the low dataset
!mkdir datasets
!mkdir datasets/offenseval

# dataset
id = '1qbg1phsd3yEg55wTmj10KyeIA5YHRQca'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/offenseval/offenseval.csv')

# direct NER
try:
  id = '1NYOKZ6t3Jo99shrTvtKzY3HjYX-zoMZF'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/offenseval/NER_embedding.npy')
except:
  print("direct NER not working")

# Generalize: GP GN GLOVE
try:
  id = '152264axxTfmuYfb_7oWYQJFCggt06CEC'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/offenseval/gpgn_embedding.pkl')
except:
  print("Generalize: GP GN GLOVE not working")

# Explicit: dict2vec
try: 
  id = '1059cRocqijTNzrl0UOXkFnngqpEZ54c1'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/offenseval/dict2vec_embedding.pkl')
except:
  print("Explicit: dict2vec not working")

# Implicit: Sarcasm
try:
  id = '1FmU_39K-qExz-mmqqFQ1Oy3KjfAldJco'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/offenseval/sarcasm_embedding.txt')
except:
  print("Implicit: Sarcasm not working")

mkdir: cannot create directory ‘datasets’: File exists


In [ ]:
import pandas as pd
df = pd.read_csv('datasets/offenseval/offenseval.csv')
df.head()

,tweet,label
0,@USER She should ask a few native Americans wh...,offensive
1,@USER @USER Go home you’re drunk!!! @USER #MAG...,offensive
2,Amazon is investigating Chinese employees who ...,not-offensive
3,"@USER Someone should'veTaken"" this piece of sh...",offensive
4,@USER @USER Obama wanted liberals &amp; illega...,not-offensive


In [ ]:
sentence_list = df["tweet"].tolist()
label_list = df['label'].tolist()
print(sentence_list[:5])
print(label_list[:5])

['@USER She should ask a few native Americans what their take on this is.', '@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL', 'Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT', '@USER Someone should\'veTaken" this piece of shit to a volcano. 😂"', '@USER @USER Obama wanted liberals &amp; illegals to move into red states']
['offensive', 'offensive', 'not-offensive', 'offensive', 'not-offensive']


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import re

tokenized_words = []
for sentence in sentence_list:
    clean_sent = re.sub(r'[^\w\s]','', sentence).lower()
    tokenize_word = word_tokenize(clean_sent)
    tokenized_words.append(tokenize_word)

length_list = [len(words) for words in tokenized_words]	
print(max(length_list))
seq_length = 64

# word index encode
from keras.preprocessing.sequence import pad_sequences

tokenPAD = "<pad>"
tokenUNK = "<unknown>"

unique_words_temp = set()
for sentence in tokenized_words:
    for word in sentence:
        unique_words_temp.add(word)
unique_words = [tokenPAD, tokenUNK]
unique_words.extend(unique_words_temp)

unique_words_dict = {word: i for i, word in enumerate(unique_words)}

word_index_encode = []
for sentence in tokenized_words:
    word_index_encode_temp = []    
    for word in sentence:
        try:
            word_index_encode_temp.append(unique_words_dict[word])
        except KeyError:
            word_index_encode_temp.append(unique_words_dict[tokenUNK])
    word_index_encode.append(word_index_encode_temp)

#Pad sequences
word_index_encode = pad_sequences(word_index_encode, maxlen = seq_length, value = unique_words_dict[tokenPAD], padding = "post")

#label encoding
from sklearn.preprocessing import LabelEncoder

labels = np.unique(label_list)

num_class = len(labels)
lEnc = LabelEncoder()
lEnc.fit(labels)

encoded_labels = lEnc.transform(label_list)
print(labels)
print(lEnc.transform(labels))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
103
['not-offensive' 'offensive']
[0 1]


In [ ]:
unique_words[:5]

['<pad>', '<unknown>', 'organized', 'coaching', 'against']

In [ ]:
dict2vec_dict = pickle.load( open( "datasets/offenseval/dict2vec_embedding.pkl", "rb" ))

NameError: ignored

In [ ]:
import pickle 

dict2vec_dict = pickle.load( open( "datasets/offenseval/dict2vec_embedding.pkl", "rb" ))

dict2vec_embedding_matrix = np.zeros((len(unique_words), 300))
for i, word in enumerate(unique_words):
    try:
        if word != tokenPAD: # set pad weights to 0
            dict2vec_embedding_matrix[i] = dict2vec_dict[word]
    except KeyError:
        pass
        
dict2vec_embedding_matrix.shape

(20766, 300)

In [ ]:
gpgn_glove_dict = pickle.load( open("datasets/offenseval/gpgn_embedding.pkl", "rb" ) )
glove_embedding_matrix = np.zeros((len(unique_words), 300))
for i, word in enumerate(unique_words):
    try:
        if word != tokenPAD: #set pad weights to 0
            glove_embedding_matrix[i] = gpgn_glove_dict[word]
    except KeyError:
        pass
        
glove_embedding_matrix.shape

(20766, 300)

###hatEval

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# download the low dataset
!mkdir datasets
!mkdir datasets/hateval

# dataset
id = '1gQp4oNRepyqe69Ut0v1K2g_wADSsFM-s'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/hateval/hateval.csv')

# direct NER
try:
  id = '1gYwL6TGW_wFa8kG6wgk1j7DMbmuQxbLS'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/hateval/NER_embedding.npy')
except:
  print("direct NER not working")

# Generalize: GP GN GLOVE
try:
  id = '152264axxTfmuYfb_7oWYQJFCggt06CEC'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/hateval/gpgn_embedding.pkl')
except:
  print("Generalize: GP GN GLOVE not working")

# Explicit: dict2vec
try: 
  id = '1059cRocqijTNzrl0UOXkFnngqpEZ54c1'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/hateval/dict2vec_embedding.pkl')
except:
  print("Explicit: dict2vec not working")

# Implicit: Sarcasm
try:
  id = '1eLgxM7K87vf_nyL5G44Zw2cdlELg64Rv'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/hateval/sarcasm_embedding.txt')
except:
  print("Implicit: Sarcasm not working")

mkdir: cannot create directory ‘datasets’: File exists


In [ ]:
dict2vec_dict2 = pickle.load( open( "datasets/hateval/dict2vec_embedding.pkl", "rb" ))

In [ ]:
dict2vec_dict == dict2vec_dict2

In [ ]:
import pandas as pd
df2 = pd.read_csv('datasets/hateval/hateval.csv')
df2.head()

,text,label
0,"Hurray, saving us $$$ in so many ways @potus @...",hateful
1,Why would young fighting age men be the vast m...,hateful
2,@KamalaHarris Illegals Dump their Kids at the ...,hateful
3,NY Times: 'Nearly All White' States Pose 'an A...,non-hateful
4,Orban in Brussels: European leaders are ignori...,non-hateful


In [ ]:
sentence_list = df2["text"].tolist()
label_list = df2['label'].tolist()
print(sentence_list[:5])
print(label_list[:5])

['Hurray, saving us $$$ in so many ways @potus @realDonaldTrump #LockThemUp #BuildTheWall #EndDACA #BoycottNFL #BoycottNike', "Why would young fighting age men be the vast majority of the ones escaping a war &amp; not those who cannot fight like women, children, and the elderly?It's because the majority of the refugees are not actually refugees they are economic migrants trying to get into Europe.... https://t.co/Ks0SHbtYqn", '@KamalaHarris Illegals Dump their Kids at the border like Road Kill and Refuse to Unite! They Hope they get Amnesty, Free Education and Welfare Illegal #FamilesBelongTogether in their Country not on the Taxpayer Dime Its a SCAM #NoDACA #NoAmnesty #SendThe', "NY Times: 'Nearly All White' States Pose 'an Array of Problems' for Immigrants https://t.co/ACZKLhdMV9 https://t.co/CJAlSXCzR6", 'Orban in Brussels: European leaders are ignoring the will of the people, they do not want migrants https://t.co/NeYFyqvYlX']
['hateful', 'hateful', 'hateful', 'non-hateful', 'non-h

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import re

tokenized_words = []
for sentence in sentence_list:
    clean_sent = re.sub(r'[^\w\s]','', sentence).lower()
    tokenize_word = word_tokenize(clean_sent)
    tokenized_words.append(tokenize_word)

length_list = [len(words) for words in tokenized_words]	
print(max(length_list))
seq_length = 64

# word index encode
from keras.preprocessing.sequence import pad_sequences

tokenPAD = "<pad>"
tokenUNK = "<unknown>"

unique_words_temp = set()
for sentence in tokenized_words:
    for word in sentence:
        unique_words_temp.add(word)
unique_words = [tokenPAD, tokenUNK]
unique_words.extend(unique_words_temp)

unique_words_dict = {word: i for i, word in enumerate(unique_words)}

word_index_encode = []
for sentence in tokenized_words:
    word_index_encode_temp = []    
    for word in sentence:
        try:
            word_index_encode_temp.append(unique_words_dict[word])
        except KeyError:
            word_index_encode_temp.append(unique_words_dict[tokenUNK])
    word_index_encode.append(word_index_encode_temp)

#Pad sequences
word_index_encode = pad_sequences(word_index_encode, maxlen = seq_length, value = unique_words_dict[tokenPAD], padding = "post")

#label encoding
from sklearn.preprocessing import LabelEncoder

labels = np.unique(label_list)

num_class = len(labels)
lEnc = LabelEncoder()
lEnc.fit(labels)

encoded_labels = lEnc.transform(label_list)
print(labels)
print(lEnc.transform(labels))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
93
['hateful' 'non-hateful']
[0 1]


In [ ]:
import pickle 

dict2vec_dict2 = pickle.load( open( "datasets/hateval/dict2vec_embedding.pkl", "rb" ))

dict2vec_embedding_matrix2 = np.zeros((len(unique_words), 300))
for i, word in enumerate(unique_words):
    try:
        if word != tokenPAD: # set pad weights to 0
            dict2vec_embedding_matrix2[i] = dict2vec_dict2[word]
    except KeyError:
        pass
        
dict2vec_embedding_matrix2.shape

In [ ]:
gpgn_glove_dict2 = pickle.load( open("datasets/hateval/gpgn_embedding.pkl", "rb" ) )
glove_embedding_matrix2 = np.zeros((len(unique_words), 300))
for i, word in enumerate(unique_words):
    try:
        if word != tokenPAD: #set pad weights to 0
            glove_embedding_matrix2[i] = gpgn_glove_dict2[word]
    except KeyError:
        pass
        
glove_embedding_matrix2.shape

###Stromfront

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# download the low dataset
!mkdir datasets
!mkdir datasets/stormfront

# dataset
id = '1LNjeEeHNpJwS4O9OK2gOSpwddwzHCaqW'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/stormfront/stormfront.csv')

# direct NER
try:
  id = '1MDMJVjS5Rn5kDSzqRQFW3bhTqmGVSse1'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/stormfront/NER_embedding.npy')
except:
  print("direct NER not working")

# Generalize: GP GN GLOVE
try:
  id = '152264axxTfmuYfb_7oWYQJFCggt06CEC'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/stormfront/gpgn_embedding.pkl')
except:
  print("Generalize: GP GN GLOVE not working")

# Explicit: dict2vec
try: 
  id = '1059cRocqijTNzrl0UOXkFnngqpEZ54c11'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/stormfront/dict2vec_embedding.pkl')
except:
  print("Explicit: dict2vec not working")

# Implicit: Sarcasm
try:
  id = '1MbwDpyCBY4j5gVsYqrq4NSWae0fhtMrR'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/stormfront/sarcasm_embedding.txt')
except:
  print("Implicit: Sarcasm not working")

mkdir: cannot create directory ‘datasets’: File exists
Explicit: dict2vec not working


In [ ]:
import pandas as pd
df = pd.read_csv('datasets/stormfront/stormfront.csv')
df.head()

,sentence,label,user_id,file_id
0,"b'As of March 13th , 2014 , the booklet had be...",noHate,572066,12834217_1
1,b'In order to help increase the booklets downl...,noHate,572066,12834217_2
2,b'( Simply copy and paste the following text i...,noHate,572066,12834217_3
3,b'Click below for a FREE download of a colorfu...,hate,572066,12834217_4
4,"b""Click on the `` DOWNLOAD ( 7.42 MB ) '' gree...",noHate,572066,12834217_5


In [ ]:
sentence_list = df["sentence"].tolist()
label_list = df['label'].tolist()
print(sentence_list[:5])
print(label_list[:5])

["b'As of March 13th , 2014 , the booklet had been downloaded over 18,300 times and counting .'", "b'In order to help increase the booklets downloads , it would be great if all Stormfronters who had YouTube accounts , could display the following text in the description boxes of their uploaded YouTube videos .'", "b'( Simply copy and paste the following text into your YouTube videos description boxes. )'", "b'Click below for a FREE download of a colorfully illustrated 132 page e-book on the Zionist-engineered INTENTIONAL destruction of Western civilization .'", 'b"Click on the `` DOWNLOAD ( 7.42 MB ) \'\' green banner link ."']
['noHate', 'noHate', 'noHate', 'hate', 'noHate']


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np

import re

tokenized_words = []
for sentence in sentence_list:
    clean_sent = re.sub(r'[^\w\s]','', sentence).lower()
    tokenize_word = word_tokenize(clean_sent)
    tokenized_words.append(tokenize_word)

length_list = [len(words) for words in tokenized_words]	
print(max(length_list))
seq_length = 64

# word index encode
from keras.preprocessing.sequence import pad_sequences

tokenPAD = "<pad>"
tokenUNK = "<unknown>"

unique_words_temp = set()
for sentence in tokenized_words:
    for word in sentence:
        unique_words_temp.add(word)
unique_words = [tokenPAD, tokenUNK]
unique_words.extend(unique_words_temp)

unique_words_dict = {word: i for i, word in enumerate(unique_words)}

word_index_encode = []
for sentence in tokenized_words:
    word_index_encode_temp = []    
    for word in sentence:
        try:
            word_index_encode_temp.append(unique_words_dict[word])
        except KeyError:
            word_index_encode_temp.append(unique_words_dict[tokenUNK])
    word_index_encode.append(word_index_encode_temp)

#Pad sequences
word_index_encode = pad_sequences(word_index_encode, maxlen = seq_length, value = unique_words_dict[tokenPAD], padding = "post")

#label encoding
from sklearn.preprocessing import LabelEncoder

labels = np.unique(label_list)

num_class = len(labels)
lEnc = LabelEncoder()
lEnc.fit(labels)

encoded_labels = lEnc.transform(label_list)
print(labels)
print(lEnc.transform(labels))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
329
['hate' 'idk/skip' 'noHate' 'relation']
[0 1 2 3]


In [ ]:
import pickle 

dict2vec_dict = pickle.load( open( "datasets/stormfront/dict2vec_embedding.pkl", "rb" ))

dict2vec_embedding_matrix = np.zeros((len(unique_words), 300))
for i, word in enumerate(unique_words):
    try:
        if word != tokenPAD: # set pad weights to 0
            dict2vec_embedding_matrix[i] = dict2vec_dict[word]
    except KeyError:
        pass
        
dict2vec_embedding_matrix.shape

FileNotFoundError: ignored

###Founta

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# download the low dataset
!mkdir datasets
!mkdir datasets/founta

# dataset
id = '1sU7UmXOmJjz6_8o_7uLICVf-cbAsCtD8'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/founta/founta.csv')

# direct NER
try:
  id = '1TbxR73VHaeXkOm1adIuPNJN74h-a1xK3'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/founta/NER_embedding.npy')
except:
  print("direct NER not working")

# Generalize: GP GN GLOVE
try:
  id = '152264axxTfmuYfb_7oWYQJFCggt06CEC'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/founta/gpgn_embedding.pkl')
except:
  print("Generalize: GP GN GLOVE not working")

# Explicit: dict2vec
try: 
  id = '1059cRocqijTNzrl0UOXkFnngqpEZ54c1'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/founta/dict2vec_embedding.pkl')
except:
  print("Explicit: dict2vec not working")

# Implicit: Sarcasm
try:
  id = '1n-Ud4m8IvXP8OEim1zk7NZwIIKILtYFd'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('datasets/founta/sarcasm_embedding.txt')
except:
  print("Implicit: Sarcasm not working")

mkdir: cannot create directory ‘datasets’: File exists
